In [ ]:
import pandas as pd
from pathlib import Path
import geopandas as gpd

In [ ]:
BASE_PATH = Path.cwd()
DATA_PATH = BASE_PATH /'data'
MAP_PATH = DATA_PATH / 'map'
CASE_PATH = DATA_PATH / 'case'

In [ ]:
path = CASE_PATH /'analyze2016.csv'
df = pd.read_csv(path)

In [ ]:
df = df[df['Year'] != 2016]

In [ ]:
df.shape[0]

In [ ]:
df

In [ ]:
grouped_df = df.groupby(['Year', 'Addr_Comm']).size().reset_index(name='count')

# Pivoting the table to get years as columns
pivot_df = grouped_df.pivot(index='Addr_Comm', columns='Year', values='count').fillna(0).reset_index()

print(pivot_df)

In [ ]:
pivot_df.to_csv(CASE_PATH /'case_after_2016.csv', encoding='utf-8-sig', index=False)

In [ ]:
cm = gpd.read_file('data/map/commune.json')
case_before_2016 = pd.read_csv(CASE_PATH /'casebycom.csv')

In [ ]:
cm['Addr_Comm'] = cm['danhTuChun'] + ' ' + cm['diaDanh']
cm['Addr_Comm'] = cm['Addr_Comm'].str.upper().str.replace('PHƯỜNG ','').str.replace('XÃ ','')
comm_name = cm[['OBJECTID', 'Addr_Comm']]

In [ ]:
case_b_2016 = pd.merge(comm_name, case_before_2016, on='OBJECTID', how='inner')

In [ ]:
case_after_2016 = pd.read_csv(CASE_PATH /'case_after_2016.csv')

In [ ]:
total_case = pd.merge(case_b_2016, case_after_2016, on='Addr_Comm', how='left').fillna(0)

In [ ]:
socio = pd.read_csv(DATA_PATH /'socio.csv')

In [ ]:
socio['Addr_Comm'] = socio['Addr_Comm'].str.replace('Phường ','').str.replace('Xã ','').str.strip().str.upper()

In [ ]:
cm_socio = cm[['OBJECTID','Addr_Comm', 'Shape_Area']]

In [ ]:
cm_socio['Shape_Area'] = cm_socio['Shape_Area']/1000000

In [ ]:
cm_socio = pd.merge(socio, cm_socio, on='Addr_Comm', how='outer')